In [1]:
import cv2
import numpy as np
from collections import deque

#Templates for Rock, Paper, and Waving Gestures
template = cv2.imread('images/handbw.png',0)
w, h = template.shape[::-1]

template2 = cv2.imread('images/fistbw2.png',0)
w2, h2 = template2.shape[::-1]

template3 = cv2.imread('images/waving.png',0)
w3, h3 = template3.shape[::-1]


def mySkinDetect(src):
    # Surveys of skin color modeling and detection techniques:
    # 1. Vezhnevets, Vladimir, Vassili Sazonov, and Alla Andreeva. "A survey on pixel-based skin color detection techniques." Proc. Graphicon. Vol. 3. 2003.
    # 2. Kakumanu, Praveen, Sokratis Makrogiannis, and Nikolaos Bourbakis. "A survey of skin-color modeling and detection methods." Pattern recognition 40.3 (2007): 1106-1122.
    dst = np.zeros((src.shape[0], src.shape[1], 1), dtype = "uint8")
    for i in range(src.shape[0]):
        for j in range(src.shape[1]):
            #b,g,r = src[i,j]
            b = int(src[i,j][0])
            g = int(src[i,j][1])
            r = int(src[i,j][2])
            if(r>95 and g>40 and b>20 and max(r,g,b)-min(r,g,b)>15 and abs(r-g)>15 and r>g and r>b):
                dst[i,j] = 255
    return dst

def myFrameDifferencing(prev, curr):
    # For more information on operation with arrays: 
    # http://docs.opencv.org/modules/core/doc/operations_on_arrays.html
    dst = cv2.absdiff(prev, curr)
    dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    _, dst = cv2.threshold(dst, 50, 255, cv2.THRESH_BINARY)
    return dst

def myMotionEnergy(mh):
    # the window of time is 3
    mh0 = mh[0]
    mh1 = mh[1]
    mh2 = mh[2]
    dst = np.zeros((mh0.shape[0], mh0.shape[1], 1), dtype = "uint8")
    for i in range(mh0.shape[0]):
        for j in range(mh0.shape[1]):
            if mh0[i,j] == 255 or mh1[i,j] == 255 or mh2[i,j] == 255:
                dst[i,j] = 255
    return dst

In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    #Thresholds
    threshold = 0.6
    threshold_d = 0.4
    #if not successful, exit program
    if not cap.isOpened():
        print("Cannot open the video cam")
        return -1
    
    # read a first frame from video
    success, prev_frame = cap.read()
    
    #if not successful, exit program
    if not success:
        print("Cannot read a frame from video stream")
        return -1
    
    
    cv2.namedWindow("frame", cv2.WINDOW_AUTOSIZE)
    
    #Setting up Prev Frame for Frame Differencing
    prev_frame = cv2.resize(prev_frame,(450,300))
    cropimage = prev_frame[50:160, 50:160]
    fMH1 = np.zeros((cropimage.shape[0], cropimage.shape[1], 1), dtype = "uint8")
    fMH2 = fMH1.copy()
    fMH3 = fMH1.copy()
    myMotionHistory = deque([fMH1, fMH2, fMH3]) 
    
    while(True):
        success, curr_frame = cap.read()
        curr_frame = cv2.resize(curr_frame,(450,300))
        if not success:
            print("Cannot read a frame from video stream")
            break
        
        #Setting a dedicated area for gesture recognition
        cv2.rectangle(curr_frame, (50, 50), (160, 160), (0, 255, 0), 0)
        crop_image = curr_frame[50:160, 50:160]
            
        # Frame differencing
        frameDest = myFrameDifferencing(cropimage, crop_image)
        #cv2.imshow('myFrameDifferencing',frameDest)

        # Visualizing motion history
        myMotionHistory.popleft()
        myMotionHistory.append(frameDest)
        myMH = myMotionEnergy(myMotionHistory)
        cv2.imshow('myMotionHistory',myMH)
    
        # Skin color detection
        mySkin = mySkinDetect(crop_image)
        cv2.imshow('mySkinDetect',mySkin)
        
        #Template Matching Formula
        res = cv2.matchTemplate(mySkin,template,cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        res2 = cv2.matchTemplate(mySkin,template2,cv2.TM_CCOEFF_NORMED)
        min_val2, max_val2, min_loc2, max_loc2 = cv2.minMaxLoc(res2)
        top_left2 = max_loc2
        bottom_right2 = (top_left2[0] + w2, top_left2[1] + h2)
        
        res3 = cv2.matchTemplate(myMH, template3, cv2.TM_CCOEFF_NORMED)
        min_val3, max_val3, min_loc2, max_loc2 = cv2.minMaxLoc(res3)
        top_left3 = max_loc2
        bottom_right3 = (top_left3[0] + w2, top_left3[1] + h2)
        
        #Recogizes Gesture if Res is Greater than Threshold for the gestures
        if len(res):
            location = np.where( res >= threshold)
            for pt in zip(*location[::-1]):
                #puting  rectangle on recognized erea 
                cv2.rectangle(curr_frame, top_left, bottom_right, (0,0,255), 2)
                cv2.putText(curr_frame, 'paper', (50,50), cv2.FONT_HERSHEY_PLAIN, 1.0, (0,0,255))
        
        if len(res2):
            location = np.where( res2 >= threshold)
            for pt in zip(*location[::-1]):
                #puting  rectangle on recognized erea 
                cv2.rectangle(curr_frame, top_left2, bottom_right2, (0,0,255), 2)
                cv2.putText(curr_frame, 'rock', (50,50), cv2.FONT_HERSHEY_PLAIN, 1.0, (0,0,255))
                
        if len(res3):
            location = np.where( res3 >= threshold_d)
            for pt in zip(*location[::-1]):
                #puting  rectangle on recognized erea 
                cv2.rectangle(curr_frame, top_left3, bottom_right3, (0,0,255), 2)
                cv2.putText(curr_frame, 'WAVING', (50,50), cv2.FONT_HERSHEY_PLAIN, 1.0, (0,0,255))
        
        #Show Frame
        cv2.imshow('frame',curr_frame)

        #Set Previous frame as old current frame
        prev_frame = curr_frame
        
        # wait for 'q' key press. If 'q' key is pressed, break loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    return 0
    

In [ ]:
if __name__ == "__main__":
    main()